In [76]:
import sys, os, json
import re, nltk
from nltk import pos_tag
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn, gensim
from sklearn.decomposition import PCA
import itertools
from gensim.corpora import Dictionary
from collections import defaultdict
import statsmodels.api as sm
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
os.chdir(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/data/')
from utils import preprocess_for_bow, experiment_result
from models.lda import LDAwrappers
from models.hdp import HDPwrapper
from models.gsdmm import MovieGroupProcessWrapper
from models.lftm import LFTMwrapper
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [78]:
model_name = "google/flan-t5-base"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [86]:
input_ids = tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True)

# Generate responses
output = model.generate(input_ids, max_length=100, num_return_sequences=1, num_beams=4)

# Decode and print the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

Science/Tech


In [91]:
def gen_topic_titles(experiment_result, exp_number):
    exp_id="exp_{}".format(exp_number)
    n=1
    for keywords in experiment_result[exp_id]['word_topic_pvalues']:
        prompt = "I have a topic described by the following keywords: [{}]. Based on the previous keywords, \
                what is this topic about?".format(", ".join(keywords['words']))
        input_ids = tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True)
        output = model.generate(input_ids, max_length=100, num_return_sequences=1, num_beams=4)
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        print('topic',n,'generated topic name: ', generated_text)
        n+=1


In [92]:
gen_topic_titles(res['lda_experiment'], 0)

topic 1 generated topic name:  Science/Tech
topic 2 generated topic name:  Science/Tech
topic 3 generated topic name:  presidential candidate donald trump gets into a video of a coronavirus show hosted by president donald biden
topic 4 generated topic name:  World
topic 5 generated topic name:  presidential election
topic 6 generated topic name:  Science/Tech
topic 7 generated topic name:  World


In [3]:
with open('experiments_dataset1.json', 'r') as fout:
    res=json.load(fout)

In [4]:
res.keys()

dict_keys(['lda_experiment', 'gsdmm_experiment', 'hdp_experiment', 'lftm_experiment'])

In [5]:
with open('experiments_dataset2.json', 'r') as fout:
    res2=json.load(fout)

In [6]:
import pandas as pd
def to_df(res):
    params=defaultdict(list)
    for exp in res.keys():
        params['c_we'].append(res[exp]['coherence_metrics']['c_we']['c_we'])
        params['c_we_std'].append(res[exp]['coherence_metrics']['c_we']['c_we_std'])
        #params['num_topics'].append(res[exp]['number_topics'])
        for p in res[exp]['hyperparameters'].keys():
            params[p].append(res[exp]['hyperparameters'][p])
    #if params['num_topics'][0]==None:
    #    del params['num_topics']
    return pd.DataFrame(params)
    #return params


In [7]:
def to_df2(res):
    params=defaultdict(list)
    for exp in res.keys():
        params['u_mass'].append(res[exp]['coherence_metrics']['u_mass']['u_mass'])
        params['u_mass_std'].append(res[exp]['coherence_metrics']['u_mass']['u_mass_std'])
        #params['num_topics'].append(res[exp]['number_topics'])
        for p in res[exp]['hyperparameters'].keys():
            params[p].append(res[exp]['hyperparameters'][p])
    #if params['num_topics'][0]==None:
    #    del params['num_topics']
    return pd.DataFrame(params)

In [8]:
def to_df3(res):
    params=defaultdict(list)
    for exp in res.keys():
        params['excl_we'].append(res[exp]['coherence_metrics']['c_we']['excl_we'])
        params['excl_we_std'].append(res[exp]['coherence_metrics']['c_we']['excl_we_std'])
        #params['num_topics'].append(res[exp]['number_topics'])
        for p in res[exp]['hyperparameters'].keys():
            params[p].append(res[exp]['hyperparameters'][p])
    #if params['num_topics'][0]==None:
    #    del params['num_topics']
    return pd.DataFrame(params)

## word embedding coherence

#### DATASET 1

In [53]:
lda=to_df(res2['hdp_experiment'])
lda.iloc[lda.nlargest(5, 'c_we').index]

,c_we,c_we_std,kappa,K,T,alpha,gamma,eta
23,0.153307,0.100293,1.027593,19,100,1.083157,1.016308,0.021277
11,0.140640,0.103504,1.000000,17,102,0.949774,1.061169,0.010000
0,0.139483,0.112894,1.056236,11,124,0.979222,1.123539,0.015063
12,0.138054,0.108530,1.102683,14,108,1.029898,1.024214,0.037757
10,0.135611,0.115917,1.000000,10,131,0.923611,0.968796,0.010000


In [55]:
with open('res.txt', 'w') as fout:
    for i in range(10):
        fout.write('\n')
        for t in res['hdp_experiment']['exp_23']['word_topic_pvalues']:
            fout.write(t['words'][i]+' & ')



In [60]:
len(res['hdp_experiment']['exp_10']['word_topic_pvalues'])

106

In [54]:
import statsmodels.api as sm
y = lda['c_we']
x=lda.drop(columns=['c_we', 'c_we_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   c_we   R-squared:                       0.701
Model:                            OLS   Adj. R-squared:                  0.637
Method:                 Least Squares   F-statistic:                     10.94
Date:                Tue, 07 Nov 2023   Prob (F-statistic):           2.86e-06
Time:                        16:06:36   Log-Likelihood:                 128.74
No. Observations:                  35   AIC:                            -243.5
Df Residuals:                      28   BIC:                            -232.6
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2011      0.039      5.151      0.0

## UMASS coherence

In [61]:
lda=to_df2(res2['hdp_experiment'])
lda.iloc[lda.nlargest(5, 'u_mass').index]

,u_mass,u_mass_std,kappa,K,T,alpha,gamma,eta
23,-5.075834,6.129790,1.027593,19,100,1.083157,1.016308,0.021277
11,-5.592157,6.774998,1.000000,17,102,0.949774,1.061169,0.010000
26,-5.655096,6.755908,1.000000,18,115,0.919477,1.041153,0.029618
12,-5.813483,6.870791,1.102683,14,108,1.029898,1.024214,0.037757
9,-5.984140,6.929608,1.013312,10,127,0.977072,1.036959,0.029136


In [66]:
len(res['hdp_experiment']['exp_9']['word_topic_pvalues'])

159

In [10]:
with open('res.txt', 'w') as fout:
    for i in range(10):
        fout.write('\n')
        for t in res2['hdp_experiment']['exp_15']['word_topic_pvalues']:
            fout.write(t['words'][i]+' & ')

In [67]:
y = lda['u_mass']
x=lda.drop(columns=['u_mass', 'u_mass_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 u_mass   R-squared:                       0.838
Model:                            OLS   Adj. R-squared:                  0.803
Method:                 Least Squares   F-statistic:                     24.14
Date:                Tue, 07 Nov 2023   Prob (F-statistic):           7.43e-10
Time:                        16:19:48   Log-Likelihood:                -16.632
No. Observations:                  35   AIC:                             47.26
Df Residuals:                      28   BIC:                             58.15
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1678      2.485      0.470      0.6

In [38]:
y = lda['u_mass_std']
x=lda.drop(columns=['u_mass', 'u_mass_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             u_mass_std   R-squared:                       0.905
Model:                            OLS   Adj. R-squared:                  0.885
Method:                 Least Squares   F-statistic:                     44.50
Date:                Tue, 07 Nov 2023   Prob (F-statistic):           4.80e-13
Time:                        11:29:26   Log-Likelihood:                -32.460
No. Observations:                  35   AIC:                             78.92
Df Residuals:                      28   BIC:                             89.81
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -7.7476      4.114     -1.883      0.0

### LDA exclusivity metric

In [68]:
lda=to_df3(res2['hdp_experiment'])
lda.iloc[lda.nsmallest(5, 'excl_we').index]

,excl_we,excl_we_std,kappa,K,T,alpha,gamma,eta
13,0.042677,0.074323,1.045834,19,194,1.027089,1.011850,0.026586
16,0.043316,0.071076,1.000000,19,174,1.003145,1.022710,0.032313
7,0.043470,0.081202,1.175820,14,197,1.014443,1.010659,0.036613
25,0.048216,0.071250,1.000000,10,188,1.030361,0.930585,0.016631
21,0.050275,0.073361,1.000000,10,173,0.958978,0.993384,0.038259


In [74]:
len(res['hdp_experiment']['exp_21']['word_topic_pvalues'])

147

In [17]:
with open('res.txt', 'w') as fout:
    for i in range(10):
        fout.write('\n')
        for t in res2['hdp_experiment']['exp_11']['word_topic_pvalues']:
            fout.write(t['words'][i]+' & ')

In [75]:
y = lda['excl_we']
x=lda.drop(columns=['excl_we', 'excl_we_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                excl_we   R-squared:                       0.750
Model:                            OLS   Adj. R-squared:                  0.697
Method:                 Least Squares   F-statistic:                     14.03
Date:                Tue, 07 Nov 2023   Prob (F-statistic):           2.57e-07
Time:                        16:26:05   Log-Likelihood:                 134.11
No. Observations:                  35   AIC:                            -254.2
Df Residuals:                      28   BIC:                            -243.3
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1596      0.033      4.767      0.0